In [ ]:
# import packages
import pandas as pd
import numpy as np
import os
import sys
import dill

In [ ]:
# 1) properties
main = '/content/drive/MyDrive/FateTrack/'
#sample = '516_D4_2_w2_training'
sampleList = ['507_D2_3_w2_training', '516_D4_2_w2_training', '516_D2_1_w2_training']
agg_location = main + 'agg/'
annotation_location = main + 'agg/'
#blocks = 9
blockList = [16, 9, 7]
featureCols = ["area", "filled_area", "bbox_area", "eccentricity", "solidity", "convex_area",
            "orientation", "major_axis_length","minor_axis_length", "euler_number", "perimeter", "extent",
            "equivalent_diameter", "feret_diameter_max", "perimeter_crofton",
            "moments_central-0-0", "moments_central-0-1", "moments_central-0-2", "moments_central-0-3",
            "moments_central-1-0", "moments_central-1-1", "moments_central-1-2", "moments_central-1-3",
            "moments_central-2-0", "moments_central-2-1", "moments_central-2-2", "moments_central-2-3",
            "moments_central-3-0", "moments_central-3-1", "moments_central-3-2", "moments_central-3-3",
            "moments_hu-0", "moments_hu-1", "moments_hu-2", "moments_hu-3", "moments_hu-4", "moments_hu-5", "moments_hu-6",
            "moments-0-0", "moments-0-1", "moments-0-2", "moments-0-3",
            "moments-1-0", "moments-1-1", "moments-1-2", "moments-1-3",
            "moments-2-0", "moments-2-1", "moments-2-2", "moments-2-3",
            "moments-3-0", "moments-3-1", "moments-3-2", "moments-3-3",
            "inertia_tensor-0-0", "inertia_tensor-0-1", "inertia_tensor-1-0", "inertia_tensor-1-1", "inertia_tensor_eigvals-0", "inertia_tensor_eigvals-1"]
nearbyFeatures = ["area", "filled_area", "bbox_area", "eccentricity", "solidity", "convex_area",
            "orientation", "major_axis_length","minor_axis_length", "perimeter", "extent",
            "equivalent_diameter", "feret_diameter_max", "perimeter_crofton"]
nearbyFeaturesExtra = ["nearby_cells", "mindist"]
timelapseFeatures = []#["area", "eccentricity", "extent", "nearby_cells"]
timelapseFeaturesExtra = []#["distance"]
timepoints = [8]
channels = ['2-cy3', '2-yfp', '2-cy5', '1-yfp', '1-cy5', 'latent']
date = '0716' #change to current date

['area', 'filled_area', 'bbox_area', 'eccentricity', 'solidity', 'convex_area', 'orientation', 'major_axis_length', 'minor_axis_length', 'perimeter', 'extent', 'equivalent_diameter', 'feret_diameter_max', 'perimeter_crofton']


In [ ]:
# combine measurement files
allMeasures = pd.DataFrame()
for block in range(blocks):
    allMeasures = pd.concat([allMeasures, pd.read_csv(agg_location + sample + '_block' + str(block) + 'of' + str(blocks) + '_output_features.csv')])
    print(all )
allMeasures.to_csv(agg_location + sample + '_measures_allblocks.csv')

NameError: ignored

In [ ]:
# 2a) load pickle object
def load_object(filename):
    with open(filename, 'rb') as red:
        tmp = dill.load(red)
    return(tmp)

In [ ]:
# 2b) functions to extract the nuclei of interest from the annotation file by block
def getNucleiOfInterest(block, blocks, sample):
    # load files
    labels = pd.read_csv(agg_location + sample + '_block' + str(block) + 'of' + str(blocks) + '_IDs.csv')
    features = pd.read_csv(agg_location + sample + '_block' + str(block) + 'of' + str(blocks) + '_input_features.csv')
    measures = pd.read_csv(agg_location + sample + '_block' + str(block) + 'of' + str(blocks) + '_output_features.csv')
    annotations = pd.read_csv(annotation_location + sample + '_block' + str(block) + 'of' + str(blocks) + '_annotations.csv')
    pklFile = load_object(main + sample + '_block' + str(block) + 'of' + str(blocks) + '.pkl')
    mask = pklFile.timelapse_nuclear_masks[8]
    # for i in range(features.shape[1]):
    #     print(features.columns[i])
    
    # filter out the cells on the border (features are inaccurate)
    initialLabels = annotations[annotations["annotation"] != "none"]
    initialLabels = initialLabels[initialLabels["yCoord"] > 50]
    initialLabels = initialLabels[initialLabels["xCoord"] > 50]
    initialLabels = initialLabels[initialLabels["yCoord"] < mask.shape[0] - 50]
    initialLabels = initialLabels[initialLabels["xCoord"] < mask.shape[1] - 50]

    # get labels of interest (Rd1_orig_label) from annotation file
    labels_Rd1 = np.asarray(initialLabels["pointID"])

    # remove duplicates (because of division)
    duplicates = list()
    for index, item in enumerate(sorted(np.asarray(labels["MasterID_8"]))):
        if index == 0:
            prev = item
        elif prev == item:
            duplicates.append(item) 
        else:
            prev = item
    labels_nodup = labels.loc[~labels["MasterID_8"].isin(duplicates)]
    labels_filtered = labels_nodup.loc[labels_nodup["Rd1_orig_label"].isin(labels_Rd1)]

    # concatenate the labels of interest to be in full string format
    time_label_list = list()
    for i in range(labels_filtered.shape[0]):
        label_string = ""
        for t in range(9):
            col = "MasterID_" + str(t)
            label_string = label_string + str(labels_filtered.iloc[i].loc[col])
            if t < 8:
                label_string = label_string + ":"
        time_label_list.append(label_string)
    HCR_label_list = labels_filtered["Rd1_orig_label"]

    # get features based on timelapse labels
    features_filtered = features[features["time8_label"].isin(time_label_list)]

    # get measures based on Rd1 labels
    measures_filtered = measures[measures["Rd1_orig_label"].isin(HCR_label_list)]

    # get ids based on non-duplicated labels
    ids = pd.DataFrame(annotations[annotations["pointID"].isin(HCR_label_list)]["annotation"])

    # append an identifier column
    identifiers = list()
    for i in range(labels_filtered.shape[0]):
        identifier = sample + "_" + str(block) + "_" + str(i)
        identifiers.append(identifier)
    identifiers = np.asarray(identifiers)
    features_filtered["identifier"] = identifiers
    measures_filtered["identifier"] = identifiers
    ids["identifier"] = identifiers

    # save files
    save_path = main + '/' + sample + '/'
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    features_filtered.to_csv(save_path + 'block' + str(block) + '_features.csv')
    measures_filtered.to_csv(save_path + 'block' + str(block) + '_measures.csv')
    ids.to_csv(save_path + 'block' + str(block) + '_id.csv')

In [ ]:
# 3a) function to get names of features
def getFeatures(featureList):
    allFeatures = list()
    featureList.extend(nearbyFeaturesExtra)
    featureList.extend(timelapseFeaturesExtra)
    for time in timepoints:
        for feat in featureList:
            featName = "time" + str(time) + "_" + feat
            allFeatures.append(featName)
        for feat in nearbyFeatures:
            featNameMax = "time" + str(time) + "_nearby_" + feat + '_max'
            featNameMin = "time" + str(time) + "_nearby_" + feat + '_min'
            allFeatures.append(featNameMax)
            allFeatures.append(featNameMin)
        for feat in timelapseFeatures:
            featName = "time" + str(time) + "_" + feat + "_dt"
            allFeatures.append(featName)
    return allFeatures

In [ ]:
# 3b) function to normalize intensity measurements (group into quantiles)
def normMeas(meas_df):
  for i in range(meas_df.shape[1]):
    q_range = np.arange(0.1, 1.1, 0.1)
    q = np.quantile(meas_df.iloc[:,i], q_range)
    new_val = 1
    for j in range(meas_df.shape[0]): 
      for k in range(10):
        if meas_df.iloc[j,i] <= q[9-k]:
          new_val = q_range[9-k]
      meas_df.iloc[j,i] = new_val
  return meas_df

In [ ]:
# 3b_2) function to normalize intensity measurements (group into quantiles and assign quantile value)
def normMeas(meas_df):
  for i in range(meas_df.shape[1]):
    q_range = np.arange(0.04, 1.04, 0.04)
    q = np.quantile(meas_df.iloc[:,i], q_range)
    new_val = 1
    for j in range(meas_df.shape[0]): 
      for k in range(25):
        if meas_df.iloc[j,i] <= q[24-k]:
          new_val = q_range[24-k]
      meas_df.iloc[j,i] = new_val
  return meas_df

In [ ]:
# 3b_3) scale intensities to between 0 and 1
def normMeas(meas_df):
  for i in range(meas_df.shape[1]):
    min_ch = min(meas_df.iloc[:,i])
    max_ch = max(meas_df.iloc[:,i])
    meas_df.iloc[:,i] = (meas_df.iloc[:,i] - min_ch) / (max_ch - min_ch)
  return meas_df

In [ ]:
# 3b_4) return raw intensities
def normMeas(meas_df):
    return meas_df

In [ ]:
# 3c) function to get channels of interest
def processMeasures(nucMeas, channelList):
  filteredMeas = pd.DataFrame()
  for channel in channelList:
    if channel == '1-cy5':
      ch = 'Rd1_CY5'
    elif channel == '1-yfp':
      ch = 'Rd1_YFP'
    elif channel == '2-cy3':
      ch = 'Rd2_CY3'
    elif channel == '2-cy5':
      ch = 'Rd2_CY5'
    elif channel == '2-yfp':
      ch = 'Rd2_YFP'
    else:
        continue
    mean_ch_name = ch + '_mean_intensity'
    # ### comment out later
    # q0 = ch + '_quantiles-0'
    # q1 = ch + '_quantiles-1'
    # q2 = ch + '_quantiles-2'
    # q3 = ch + '_quantiles-3'
    # q4 = ch + '_quantiles-4'
    # colList = [mean_ch_name, q0, q1, q2, q3, q4]
    # filteredMeas = pd.concat([filteredMeas, pd.DataFrame(nucMeas[colList])])
    filteredMeas = pd.concat([filteredMeas, pd.DataFrame(nucMeas[mean_ch_name])], axis = 1)
  return filteredMeas

In [ ]:
# 4) function to aggregate data from blocks and extract for ML
def prepareData(prefix, sampleList, blockList, all = False):
    allIDs = pd.DataFrame()
    allFeats = pd.DataFrame()
    allMeas = pd.DataFrame()
    allSampleNames = pd.DataFrame()

    i = 0
    for sample in sampleList:
        # aggregate features and measures
        features = pd.DataFrame()
        measures = pd.DataFrame()
        ids = pd.DataFrame()
        for b in range(blockList[i]):
            block_features = pd.read_csv(main + sample + '/' + 'block' + str(b) + '_features.csv')
            block_measures = pd.read_csv(main + sample + '/' + 'block' + str(b) + '_measures.csv')
            block_ids = pd.read_csv(main + sample + '/' + 'block' + str(b) + '_id.csv')
            features = features.append(block_features)
            measures = measures.append(block_measures)
            ids = ids.append(block_ids)
        unnormMeasures = pd.concat([pd.DataFrame(measures["identifier"]), processMeasures(measures, channels)], axis = 1)
        measures = pd.concat([pd.DataFrame(measures["identifier"]), normMeas(processMeasures(measures, channels))], axis = 1)
        sampleCol = pd.DataFrame([sample] * ids.shape[0])
        sampleNamesIDs = (pd.DataFrame(ids["identifier"])).reset_index()
        sampleCol["identifier"] = sampleNamesIDs["identifier"]
        allIDs = pd.concat([allIDs, ids])
        allFeats = pd.concat([allFeats, features])
        allMeas = pd.concat([allMeas, measures])
        allSampleNames = pd.concat([allSampleNames, sampleCol])
        i += 1

    # sample cells from channels
    if (all):
        sampledIDs = allIDs
        sampledFeats = allFeats
        sampledMeas = allMeas
        sampledSNames = allSampleNames
    else:
        minCells = sys.maxsize
        for channel in channels:
            chanIDs = allIDs[allIDs["annotation"] == channel]
            nrow = chanIDs.shape[0]
            if nrow < minCells:
                minCells = nrow
        sampledIDs = pd.DataFrame()
        sampledFeats = pd.DataFrame()
        sampledMeas = pd.DataFrame()
        sampledSNames = pd.DataFrame()
        for channel in channels:
            chanIDs = allIDs[allIDs["annotation"] == channel]
            if chanIDs.shape[0] > minCells:
                chanIDs = chanIDs.sample(minCells, replace = False)
            sampledIdentifiers = chanIDs["identifier"]
            chanFeats = allFeats[allFeats["identifier"].isin(sampledIdentifiers)]
            chanMeas = allMeas[allMeas["identifier"].isin(sampledIdentifiers)]
            chanSNames = allSampleNames[allSampleNames["identifier"].isin(sampledIdentifiers)]
            sampledIDs = pd.concat([sampledIDs, chanIDs])
            sampledFeats = pd.concat([sampledFeats, chanFeats])
            sampledMeas = pd.concat([sampledMeas, chanMeas])
            sampledSNames = pd.concat([sampledSNames, chanSNames])

    # get the important columns
    sampledIDs = sampledIDs["annotation"]
    featsOfInterest = getFeatures(featureCols)
    sampledFeats = sampledFeats[featsOfInterest]
    sampledMeas = sampledMeas.drop(labels = "identifier", axis = 1)
    sampledSNames = sampledSNames.drop(labels = "identifier", axis = 1)
    unnormMeasures = unnormMeasures.drop(labels = "identifier", axis = 1)

    # save files
    channelString = ""
    for channel in channels:
        channelString = channelString + "_" + channel
        channelString.replace("-", "")
    save_path = main + '/' + date + '/'
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    featsOfInterest = pd.DataFrame(featsOfInterest)
    featsOfInterest.to_csv(save_path + 'features' + date + prefix + channelString + '.csv')
    sampledIDs.to_csv(save_path + 'nuclei_id_' + date + prefix + channelString + '.csv', header = False, index=False)
    sampledFeats.to_csv(save_path + 'nuclei_feat_' + date + prefix + channelString + '.csv', header = False, index=False)
    sampledMeas.to_csv(save_path + 'nuclei_meas_' + date + prefix + channelString + '.csv', header = False, index=False)
    sampledSNames.to_csv(save_path + 'nuclei_samples_' + date + prefix + channelString + '.csv', header = False, index = False)
    # unnormMeasures.to_csv(save_path + 'unnorm_meas' + date + prefix + channelString + '.csv', header = False, index = False)

In [ ]:
# 5) run
i = 0
for s in sampleList:
    totalBlocks = blockList[i]
    for b in range(totalBlocks):
        getNucleiOfInterest(b, totalBlocks, s)
    i += 1
s = sampleList[2]
totalBlocks = 9
for b in range(totalBlocks):
    getNucleiOfInterest(b, totalBlocks, s)

prefix = '_allNuc_q25' # comes after data but before channel names
prepareData(prefix, sampleList, blockList, all = True)
